# Google Cloud Document AI

Document AI 是一个来自 Google Cloud 的文档理解平台，可将文档中的非结构化数据转换为结构化数据，从而更易于理解、分析和使用。

了解更多：

- [Document AI 概览](https://cloud.google.com/document-ai/docs/overview)
- [Document AI 视频和实验](https://cloud.google.com/document-ai/docs/videos)
- [立即试用！](https://cloud.google.com/document-ai/docs/drag-and-drop)

该模块包含一个基于 Google Cloud DocAI 的 `PDF` 解析器。

您需要安装两个库才能使用此解析器：

In [ ]:
%pip install --upgrade --quiet  langchain-google-community[docai]

首先，您需要设置一个 Google Cloud Storage (GCS) 存储桶并创建自己的光学字符识别 (OCR) 处理器，具体步骤如下：https://cloud.google.com/document-ai/docs/create-processor

`GCS_OUTPUT_PATH` 应为 GCS 上某个文件夹的路径（以 `gs://` 开头），而 `PROCESSOR_NAME` 应类似于 `projects/PROJECT_NUMBER/locations/LOCATION/processors/PROCESSOR_ID` 或 `projects/PROJECT_NUMBER/locations/LOCATION/processors/PROCESSOR_ID/processorVersions/PROCESSOR_VERSION_ID`。您可以通过编程方式获取它，或者从 Google Cloud Console 的“处理器详情”标签页中的“预测端点”部分复制。

In [2]:
GCS_OUTPUT_PATH = "gs://BUCKET_NAME/FOLDER_PATH"
PROCESSOR_NAME = "projects/PROJECT_NUMBER/locations/LOCATION/processors/PROCESSOR_ID"

In [1]:
from langchain_core.document_loaders.blob_loaders import Blob
from langchain_google_community import DocAIParser

现在，创建一个 `DocAIParser`。

In [3]:
parser = DocAIParser(
    location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
)

对于这个示例，您可以使用上传到公共 GCS 存储桶的 Alphabet 财报。

[2022Q1_alphabet_earnings_release.pdf](https://storage.googleapis.com/cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2022Q1_alphabet_earnings_release.pdf)

将文档传递给 `lazy_parse()` 方法以

In [4]:
blob = Blob(
    path="gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2022Q1_alphabet_earnings_release.pdf"
)

我们将每页获得一份文档，总共 11 份：

In [8]:
docs = list(parser.lazy_parse(blob))
print(len(docs))

11


您可以逐个运行 blob 的端到端解析。如果您有许多文档，也许将它们批量处理在一起，甚至将解析与处理解析结果分离开来，会是更好的方法。

In [9]:
operations = parser.docai_parse([blob])
print([op.operation.name for op in operations])

['projects/543079149601/locations/us/operations/16447136779727347991']


您可以检查操作是否完成：

In [10]:
parser.is_running(operations)

True

然后，当他们完成时，你可以解析结果：

In [11]:
parser.is_running(operations)

False

In [12]:
results = parser.get_results(operations)
print(results[0])

DocAIParsingResults(source_path='gs://vertex-pgt/examples/goog-exhibit-99-1-q1-2023-19.pdf', parsed_path='gs://vertex-pgt/test/run1/16447136779727347991/0')


现在我们可以根据解析结果生成文档了：

In [15]:
docs = list(parser.parse_from_results(results))

In [16]:
print(len(docs))

11
